<a href="https://colab.research.google.com/github/dharalakshmi/NNDL/blob/main/lab%2011%2C12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import zipfile
import os

# Upload the dataset
uploaded = files.upload()

# Unzip the uploaded file
zip_path = next(iter(uploaded))  # Gets uploaded file name
extract_dir = "neonatal_jaundice_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List directories
os.listdir(extract_dir)


Saving Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation (1).zip to Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation (1).zip


['Neonatal Jaundice Assessment A Dataset of Forehead and Sternum Images for Bilirubin Estimation']

In [39]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Build a reusable model builder
def build_model():
    inp = Input(shape=(128, 128, 3))
    x = Conv2D(32, (3, 3), activation='relu')(inp)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1)(x)

    model = Model(inputs=inp, outputs=out)
    return model

# Define optimizer list
opt_list = {
    "Adam": Adam(),
    "SGD": SGD(learning_rate=0.01, momentum=0.9),
    "RMSprop": RMSprop()
}

print("📊 Starting training with different optimizers...\n")

optimizer_results = {}

for name, optimizer in opt_list.items():
    print(f"🔧 Using Optimizer: {name}")

    model = build_model()  # Rebuild fresh model each time
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    history = model.fit(X_train, y_train, epochs=5, verbose=0, validation_split=0.1)

    final_loss = history.history['val_loss'][-1]
    final_mae = history.history['val_mae'][-1]

    print(f"📉 Final Val Loss: {final_loss:.4f} | Final Val MAE: {final_mae:.4f}\n")

    optimizer_results[name] = {
        'val_loss': final_loss,
        'val_mae': final_mae
    }

print("✅ Training complete for all optimizers.")

📊 Starting training with different optimizers...

🔧 Using Optimizer: Adam
📉 Final Val Loss: 66.6058 | Final Val MAE: 7.5277

🔧 Using Optimizer: SGD
📉 Final Val Loss: nan | Final Val MAE: nan

🔧 Using Optimizer: RMSprop
📉 Final Val Loss: 29.1756 | Final Val MAE: 4.5609

✅ Training complete for all optimizers.


In [40]:
from tensorflow.keras.layers import SimpleRNN

# Example of basic RNN model
input_rnn = Input(shape=(10, 128))  # 10 time steps, 128 features
x = SimpleRNN(64, activation='tanh')(input_rnn)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)

rnn_model = Model(inputs=input_rnn, outputs=output)
rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.summary()


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,465 (56.50 KB)

 Trainable params: 14,465 (56.50 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import numpy as np

# Reshape: Treat each 128x128 image as a sequence of 128 time steps with 128 features
X_rnn_train = X_train.reshape((X_train.shape[0], 128, 128*3))
X_rnn_test = X_test.reshape((X_test.shape[0], 128, 128*3))

model_rnn = Sequential([
    SimpleRNN(64, activation='tanh', input_shape=(128, 128*3)),
    Dense(1)
])
model_rnn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_rnn.summary()

# Train
history_rnn = model_rnn.fit(X_rnn_train, y_train, epochs=5, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)        │ (None, 64)             │        28,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,801 (112.50 KB)

 Trainable params: 28,801 (112.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - loss: 72.3163 - mae: 7.7233 - val_loss: 20.0579 - val_mae: 3.8497
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 16.9575 - mae: 3.3439 - val_loss: 11.7023 - val_mae: 2.7267
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 11.5811 - mae: 2.7609 - val_loss: 10.1441 - val_mae: 2.4914
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 10.4272 - mae: 2.7217 - val_loss: 8.9847 - val_mae: 2.2956
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 9.3959 - mae: 2.5923 - val_loss: 8.2961 - val_mae: 2.1867


In [42]:
#lab12

In [43]:
from tensorflow.keras.layers import Bidirectional

model_birnn = Sequential([
    Bidirectional(SimpleRNN(64), input_shape=(128, 128*3)),
    Dense(1)
])
model_birnn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_birnn.summary()
model_birnn.fit(X_rnn_train, y_train, epochs=5, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 128)            │        57,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,601 (225.00 KB)

 Trainable params: 57,601 (225.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - loss: 50.1250 - mae: 6.1624 - val_loss: 6.5903 - val_mae: 2.1810
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 9.9304 - mae: 2.6777 - val_loss: 7.4292 - val_mae: 2.3789
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 9.7846 - mae: 2.6864 - val_loss: 6.6171 - val_mae: 2.1613
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 8.1614 - mae: 2.4607 - val_loss: 6.6924 - val_mae: 2.0673
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - loss: 8.2364 - mae: 2.5251 - val_loss: 6.9916 - val_mae: 2.0587


In [44]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential([
    LSTM(64, input_shape=(128, 128*3)),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_lstm.summary()
model_lstm.fit(X_rnn_train, y_train, epochs=5, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │       114,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115,009 (449.25 KB)

 Trainable params: 115,009 (449.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - loss: 66.2481 - mae: 7.4618 - val_loss: 22.2122 - val_mae: 4.0728
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - loss: 17.9953 - mae: 3.4070 - val_loss: 10.3043 - val_mae: 2.5199
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - loss: 10.2812 - mae: 2.6400 - val_loss: 8.0803 - val_mae: 2.1520
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - loss: 8.9613 - mae: 2.6258 - val_loss: 7.4248 - val_mae: 2.0675
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step - loss: 8.1488 - mae: 2.4763 - val_loss: 7.1248 - val_mae: 2.0608
